In [2]:

import logging
import tensorflow as tf
import json
import os
import numpy as np
import time
import timeit
from tqdm import tqdm
from sklearn.metrics.pairwise import cosine_similarity, pairwise_distances
from scipy import sparse
import scipy.sparse as sp
from scipy.sparse import csr_matrix
from scipy.sparse import vstack, kron
import pickle
# tf.enable_eager_execution() 
from multiprocessing import Process,process, Pool, cpu_count, Manager
import threading
import networkx as nx
import matplotlib.pyplot as plt

## old_way to show the logic

In [ ]:
arr1 = np.array([1, 2, 3])
arr2 = np.array([4, 5, 6])
arr3 = np.array([1,2,3])
arr4 = np.array([7,3,1])
arr5 = np.array([1,9,6])
res_matrix = np.vstack([arr1, arr2, arr3, arr4, arr5])

tmp = list(enumerate(res_matrix))
tmp

In [ ]:
def cos_dist(vec1,vec2):
    """
    :param vec1:
    :param vec2:
    :return: the similarity between two vectors
    """
    dist1 = float(np.dot(vec1,vec2)/(np.linalg.norm(vec1)*np.linalg.norm(vec2)))
    return dist1

In [ ]:
cos_dist(res_matrix[0],res_matrix[1] )

In [ ]:
# def coculate_similarity(index, vs_mat, dict_node_neighbor):
#     print("exec")
#     node_neighbor_list = []
#     if cos_dist(vs_mat[index-1], vs_mat[index]) >= 0.5:
#         print("idx:", index,"vs_mat[index-1] =>", vs_mat[index-1], "vs_mat[index] =>", vs_mat[index])
#         node_neighbor_list.append(index)
#     dict_node_neighbor[node_idx] = node_neighbor_list

In [ ]:
def threadPool():
    """
        
    """
    mat = sp.dok_matrix((res_matrix.shape[0],res_matrix.shape[0]), dtype=np.int8) # 73309 means number of articles
    dict_node_neighbor = {}
    poolsize = 12
    pool = Pool(poolsize)
    
    for idx, val in tmp[1:]: 
        dict = {}
        dict['idx'] = idx
        dict['val'] = val
        dict['tmp'] = tmp[idx+1:]
        dict['dict_node_neighbor'] = dict_node_neighbor
        requests = pool.map(coculate_similarity, dict)
    
#     # start thread
#         for idx, val in tmp[1:]:
# #         threading.Thread(target=coculate_similarity, args=(idx, val, tmp[idx+1:], dict_node_neighbor)).start()
#             threads[i] = threading.Thread(target=coculate_similarity, args=(idx, val, tmp[idx+1:], dict_node_neighbor))
#             threads[i].start()
#     #wait finish
#     for i in range(len(threads)):
#         threads[i].join()
    pool.wait()
    for user_id, movie_ids in dict_node_neighbor.items():
        mat[user_id, movie_ids] = 1
    
    print("d:",dict_node_neighbor )
    return mat.transpose().tocsr()

In [ ]:
# 保存紀念
# def threads():
#     """
#        cpu-comsuming 
#     """
#     mat = sp.dok_matrix((res_matrix.shape[0],res_matrix.shape[0]), dtype=np.int8) # 73309 means number of articles
#     dict_node_neighbor = {}
    
# #     # start thread
#         for idx, val in tmp[1:]:
# #         threading.Thread(target=coculate_similarity, args=(idx, val, tmp[idx+1:], dict_node_neighbor)).start()
#             threads[i] = threading.Thread(target=coculate_similarity, args=(idx, val, tmp[idx+1:], dict_node_neighbor))
#             threads[i].start()
# #     #wait finish
#     for i in range(len(threads)):
#         threads[i].join()
#     for user_id, movie_ids in dict_node_neighbor.items():
#         mat[user_id, movie_ids] = 1
    
#     print("d:",dict_node_neighbor )
#     return mat.transpose().tocsr()

In [ ]:
# 沒用
# def double_loop():
#     dict_node_neighbor = Manager().dict()
#     mat = sp.dok_matrix((res_matrix.shape[0],res_matrix.shape[0]), dtype=np.int8) # 73309 means number of articles
#     pool = Pool(11) # on 8 processors
#     try:
#         for idx in range(1,len(res_matrix)):
#             pool.apply_async(coculate_similarity, (idx, res_matrix[idx+1:], dict_node_neighbor))
#         pool.close()
#         pool.join()
#     finally:
#         #convert
#         print(dict_node_neighbor)
#         for user_id, movie_ids in dict_node_neighbor.items():
#             mat[user_id, movie_ids] = 1
#         return mat.transpose().tocsr()

In [ ]:
# get list
tmp_keys = [idx for idx, _ in tmp[:]]
for i in tmp_keys[1:]:
    print(i)

In [ ]:
def coculate_similarity(idx, tmp):
    node_neighbor_list = []
    arch_idx = idx
    arch_val = tmp[idx]
    
    for next_idx, next_val in tmp[idx+1:]:
        if cos_dist(arch_val, next_val) >= 0.5:
            node_neighbor_list.append(next_idx)
    
    dict_node_neighbor[arch_idx] = node_neighbor_list

In [ ]:
def double_loop():
    mat = sp.dok_matrix((res_matrix.shape[0],res_matrix.shape[0]), dtype=np.int8) # 73309 means number of articles
    dict_node_neighbor = {}
    #
    for i in tmp_keys[1:]:
        coculate_similarity(i, tmp[i:])
    #all thread finished
    for user_id, movie_ids in dict_node_neighbor.items():
        mat[user_id, movie_ids] = 1
        
    return mat.transpose().tocsr()

In [ ]:
if __name__ == '__main__':
    # Protected main function
    threadPool()

In [ ]:
###Backup
# def threads():
#     """
#         效果顯著，可是我們需要固定size的pool
#     """
#     mat = sp.dok_matrix((res_matrix.shape[0],res_matrix.shape[0]), dtype=np.int8) # 73309 means number of articles
#     dict_node_neighbor = {}
#     threads = [None] * 20
    
#     # start thread
#     for i in tqdm(range(len(threads))):
#         for idx, val in tmp[1:]:
# #         threading.Thread(target=coculate_similarity, args=(idx, val, tmp[idx+1:], dict_node_neighbor)).start()
#             threads[i] = threading.Thread(target=coculate_similarity, args=(idx, val, tmp[idx+1:], dict_node_neighbor))
#             threads[i].start()
#     #wait finish
#     for i in range(len(threads)):
#         threads[i].join()
    
#     for user_id, movie_ids in dict_node_neighbor.items():
#         mat[user_id, movie_ids] = 1
    
#     print("d:",dict_node_neighbor )
#     return mat.transpose().tocsr()

In [ ]:
## backup
# def double_loop():
#     mat = sp.dok_matrix((res_matrix.shape[0],res_matrix.shape[0]), dtype=np.int8) # 73309 means number of articles
#     for idx, val in tmp[1:]:
#         for idx_n, val_n in tmp[idx+1:]:
#             if cos_dist(val, val_n) >= 0.5:
#                 mat[idx, idx_n] = 1
#     return mat.transpose().tocsr()

In [ ]:
%timeit threads()

In [ ]:
    mat = sp.dok_matrix((73309,73309), dtype=np.int8) # 73309 means number of articles
    for article_id, neighbor_ids in tqdm(desired_convert_dict.items() ,desc="dict -> sparse matrix"):
        mat[article_id, neighbor_ids] = 1
    mat = mat.transpose().tocsr()

    # write sparse matrix
    # s_m_path = "/home/hengshiou/Documents/chameleon_recsys/adressa" + "/graph/sparse_matrix.npz"
    s_m_path = FLAGS.output_sparse_matrix_resources_path
    sp.save_npz(s_m_path, mat)

## Optimized_way

In [ ]:
arr1 = np.array([1, 2, 3, 5])
arr2 = np.array([4, 5, 6, 0])
arr3 = np.array([1,2,3,0])
arr4 = np.array([0,1,0,0])
arr5 = np.array([1,9,6,9])
res_matrix = np.vstack([arr1, arr2, arr3, arr4, arr5])
res_matrix

In [ ]:
cos_dist(res_matrix[1], res_matrix[4])

In [ ]:
# 比較 row 的方式
similarity_rows = cosine_similarity(res_matrix[1].reshape(1, -1), res_matrix[1:], dense_output=False)
print(similarity_rows)
pass_threshold_rows = np.where(similarity_rows >= 0.5, 1, 0)
print(pass_threshold_rows)

# 除了第一個不要，後面為 1 的放入到 list 當中

In [ ]:
cosine_similarity(res_matrix[2].reshape(1, -1), res_matrix[2:], dense_output=False)

In [ ]:
cosine_similarity(res_matrix[3].reshape(1, -1), res_matrix[3:], dense_output=False)

In [ ]:
similarity_matrix = cosine_similarity(res_matrix[1:], dense_output=True)
similarity_matrix


In [ ]:
A = np.array([[1,2],[3,4]])
print(A.shape)
np.pad(A, ((0,0),(2,0)), 'constant')

In [ ]:
# iterate res_matrix 先
def opt_way():
    centroids = []
    for idx in range(1, len(res_matrix[:])):
        # 
        similarity_rows = cosine_similarity(res_matrix[idx].reshape(1, -1), res_matrix[idx:], dense_output=False)

        # rm self-loop
        similarity_rows[0][0] = 0

        # add padding
        neighbor = np.where(similarity_rows >= 0.5, 1, 0)
        zeros = np.zeros(idx-1, dtype=np.int64)
        neighbor = np.append(zeros, neighbor)

        centroids = vstack((centroids, csr_matrix(neighbor)), format='csr')

    sp.save_npz('sparse_matrix_qw.npz', centroids)

# centroids.todense()

In [ ]:
%timeit opt_way()

In [ ]:
sparse_matrix = sp.load_npz('sparse_matrix_q.npz')
sparse_matrix.todense()

In [ ]:
sparse_matrixw = sp.load_npz('sparse_matrix_qw.npz')
sparse_matrixw.todense()

In [ ]:
# 去除 self-loop
np.fill_diagonal(similarity_matrix, 0)
similarity_matrix

In [ ]:
# 去除小於 threshold, 將留下的轉成 1,0
pass_threshold_matrix = np.where(similarity_matrix >= 0.5, 1, 0)
pass_threshold_matrix

In [ ]:
sparse.csr_matrix(pass_threshold_matrix)

In [ ]:
#這邊要進行優畫版本
def opt_way():
    # 取得 similarity_matrix
    similarity_matrix = cosine_similarity(res_matrix[1:], dense_output=True)
    # 去除 self-loop
    np.fill_diagonal(similarity_matrix, 0)
    # 去除小於 threshold, 將留下的轉成 1,0
    pass_threshold_matrix = np.where(similarity_matrix >= 0.5, 1, 0)
    # 轉化成 csr_matrix
    mat = sparse.csr_matrix(pass_threshold_matrix)
    #write sparse_matrix
    sparse.save_npz("sparse_matrix.npz", mat)
    return mat

In [ ]:
print(opt_way().shape)

In [ ]:
%timeit opt_way()

## TF.version_similarity

In [3]:
arr1 = np.array([1, 2, 3, 5])
arr2 = np.array([4, 5, 6, 0])
arr3 = np.array([1,2,3,0])
arr4 = np.array([7,3,1,1])
arr5 = np.array([1,9,6,9])
res_matrix = np.vstack([arr1, arr2, arr3, arr4, arr5]).astype(np.float32)
type(res_matrix)

numpy.ndarray

In [ ]:
def compute_cosine_distances(a, b):
    # x shape is n_a * dim
    # y shape is n_b * dim
    # results shape is n_a * n_b

    normalize_a = tf.nn.l2_normalize(a,1)        
    normalize_b = tf.nn.l2_normalize(b,1)
    distance = 1 - tf.matmul(normalize_a, normalize_b, transpose_b=True)
    return distance

In [ ]:
# 對照組
cosine_similarity(res_matrix[1:], dense_output=False)

In [ ]:
similarity_matrix = 1 - compute_cosine_distances(res_matrix[1], res_matrix[1:]).numpy()
similarity_matrix

In [ ]:
 tf.executing_eagerly()

## 將 imput 轉成 tf.tensor 進行測試

In [35]:
res_matrix
data_tf = tf.convert_to_tensor(res_matrix, np.float32)
data_tf.shape

TensorShape([Dimension(5), Dimension(4)])

In [25]:
np.random.random_integers(0, 1, (3, 3))

/usr/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: This function is deprecated. Please call randint(0, 1 + 1) instead
  """Entry point for launching an IPython kernel.


array([[1, 0, 1],
       [0, 0, 0],
       [0, 1, 0]])

In [115]:
#全部比較法
#@https://stackoverflow.com/questions/57612543/how-to-perform-similarity-function-over-columns-of-a-tensor-in-tensorflow
# Normalize the columns of the tensor
normalized_tensor = tf.math.l2_normalize(data_tf, axis=0)

# Get the dot product between the columns
scores = tf.matmul(normalized_tensor, normalized_tensor, transpose_a=True)

zero_diag = scores - tf.linalg.diag(tf.linalg.diag_part(scores))
triangular = tf.matrix_band_part(zero_diag, 0, -1)
# 練習使用 tf.where https://stackoverflow.com/questions/48909198/tensorflow-conditions-check-if-the-values-inside-the-tensor-is-zero-or-greater
a = tf.ones_like(triangular)
b = tf.zeros_like(triangular)
whered_tensor = tf.where(tf.greater(triangular, 0.5), a, b)


with tf.Session() as sess:
#     v = sess.run()
    print(whered_tensor)
    print(whered_tensor.eval())
    w1 = tf.size(whered_tensor).eval()
    print(w1)
    
w1



Tensor("Select_44:0", shape=(4, 4), dtype=float32)
[[0. 1. 1. 0.]
 [0. 0. 1. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 0.]]
16


16

## test networkx

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
# 放入到 graph
#https://stackoverflow.com/questions/26665799/networkx-adding-edges-to-a-graph-from-a-dictionary-with-lists-as-values

G = nx.from_dict_of_lists(dict)
G.nodes()

In [ ]:
G.edges(1)

In [ ]:
[n for n in G.neighbors(1)]

In [ ]:
A = nx.to_scipy_sparse_matrix(G)
oa = nx.adjacency_matrix(G)
I = np.matrix(np.eye(A.shape[0]))
X = np.matrix([
            [i, -i]
            for i in range(A.shape[0])
        ], dtype=float)
feature = X.tolil()
A_hat = A + I

In [ ]:
A.shape, I, X

In [ ]:
nx.draw(G, with_labels=True)
plt.show()

In [ ]:
print(oa == A)

## 建立 adj_matrix, feature, 最後透過 GCN 跑


In [ ]:
# 讀取 csr_matrix, 也就是 adj matrix 概念
padding_first_sparse_matrixw = sp.load_npz('sparse_matrix_qw.npz')
adj = vstack(padding_first_sparse_matrixw[1:], format='csr')
adj.todense(), type(adj)

In [ ]:
# 建立 feature 出來
# e.g. features = sp.vstack((allx, tx)).tolil()
rows = adj.shape[0]
cols = adj.shape[1]
feat_x = sparse.csr_matrix(np.random.random((rows, cols)))
feat_x.todense(), type(feat_x)

In [ ]:
G = nx.from_scipy_sparse_matrix(adj)
nx.draw(G, with_labels=True)
plt.show()

### 定義 TF.graph.layer

In [ ]:
import tensorflow as tf


def matmul(x, y, sparse=False):
    """Wrapper for sparse matrix multiplication."""
    if sparse:
        return tf.sparse_tensor_dense_matmul(x, y)
    return tf.matmul(x, y)


class GraphConvLayer:
    def __init__(
            self,
            input_dim,
            output_dim,
            activation=None,
            use_bias=False,
            name="graph_conv"):
        """Initialise a Graph Convolution layer.
        Args:
            input_dim (int): The input dimensionality.
            output_dim (int): The output dimensionality, i.e. the number of
                units.
            activation (callable): The activation function to use. Defaults to
                no activation function.
            use_bias (bool): Whether to use bias or not. Defaults to `False`.
            name (str): The name of the layer. Defaults to `graph_conv`.
        """
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.activation = activation
        self.use_bias = use_bias
        self.name = name

        with tf.variable_scope(self.name):
            self.w = tf.get_variable(
                name='w',
                shape=(self.input_dim, self.output_dim),
                initializer=tf.initializers.glorot_uniform())

            if self.use_bias:
                self.b = tf.get_variable(
                    name='b',
                    initializer=tf.constant(0.1, shape=(self.output_dim,)))

    def call(self, adj_norm, x, sparse=False):
        x = matmul(x=x, y=self.w, sparse=sparse)  # XW
        x = matmul(x=adj_norm, y=x, sparse=True)  # AXW

        if self.use_bias:
            x = tf.add(x, self.use_bias)          # AXW + B

        if self.activation is not None:
            x = self.activation(x)                # activation(AXW + B)

        return x

    def __call__(self, *args, **kwargs):
        return self.call(*args, **kwargs)

In [ ]:
def sparse_to_tuple(sparse_mx):
    """Convert sparse matrix to tuple representation."""
    # The zeroth element of the tuple contains the cell location of each
    # non-zero value in the sparse matrix
    # The first element of the tuple contains the value at each cell location
    # in the sparse matrix
    # The second element of the tuple contains the full shape of the sparse
    # matrix
    def to_tuple(mx):
        if not sp.isspmatrix_coo(mx):
            mx = mx.tocoo()
        coords = np.vstack((mx.row, mx.col)).transpose()
        values = mx.data
        shape = mx.shape
        return coords, values, shape

    if isinstance(sparse_mx, list):
        for i in range(len(sparse_mx)):
            sparse_mx[i] = to_tuple(sparse_mx[i])
    else:
        sparse_mx = to_tuple(sparse_mx)

    return sparse_mx

In [ ]:
# TensorFlow placeholders
ph = {
    'adj_norm': tf.sparse_placeholder(tf.float32, name="adj_mat"),
    'x': tf.sparse_placeholder(tf.float32, name="x")
}

# out_dims_in_each_layer
l_sizes = [200, 100, 50]

o_fc1 = GraphConvLayer(
    input_dim=feat_x.shape[-1],
    output_dim=l_sizes[0],
    name='fc1',
    activation=tf.nn.tanh)(adj_norm=ph['adj_norm'], x=ph['x'], sparse=True)

o_fc2 = GraphConvLayer(
    input_dim=l_sizes[0],
    output_dim=l_sizes[1],
    name='fc2',
    activation=tf.nn.tanh)(adj_norm=ph['adj_norm'], x=o_fc1)

o_fc3 = GraphConvLayer(
    input_dim=l_sizes[1],
    output_dim=l_sizes[2],
    name='fc3',
    activation=tf.nn.tanh)(adj_norm=ph['adj_norm'], x=o_fc2)



In [ ]:
tf.test.is_built_with_cuda()

In [ ]:
config = tf.ConfigProto(
#     device_count = {'GPU': 0}
)
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)
sess.run(tf.global_variables_initializer())

adj_tilde = adj + np.identity(n=adj.shape[0])
d_tilde_diag = np.squeeze(np.sum(np.array(adj_tilde), axis=1))
d_tilde_inv_sqrt_diag = np.power(d_tilde_diag, -1/2)
d_tilde_inv_sqrt = np.diag(d_tilde_inv_sqrt_diag)
adj_norm = np.dot(np.dot(d_tilde_inv_sqrt, adj_tilde), d_tilde_inv_sqrt)
adj_norm_tuple = sparse_to_tuple(sp.coo_matrix(adj_norm))
feat_x_tuple = sparse_to_tuple(sp.coo_matrix((feat_x)))

feed_dict = {ph['adj_norm']: adj_norm_tuple,
             ph['x']: feat_x_tuple
            }

outputs = sess.run(o_fc3, feed_dict=feed_dict)
# x_min, x_max = outputs[:, 0].min(), outputs[:, 0].max()
# y_min, y_max = outputs[:, 1].min(), outputs[:, 1].max()


## 學習 mupliprocess

In [ ]:
from multiprocessing import Process, Manager, Pool

def f(x, tmp):
    return {x: [x*x, x*x*x]}


In [ ]:
d = dict()
pool = Pool(processes=10)
results = []
# print "[0, 1, 4,..., 81]"

for i in range(10):
    results.append(pool.apply_async(f, args=(i,tmp)))
    
pool.close()
pool.join()

for result in results:
    d.update(result.get())

d.get(9)


### 正式改造

In [ ]:
from collections import OrderedDict
import random

In [ ]:
def cos_dist(vec1,vec2):
    """
    :param vec1:
    :param vec2:
    :return: the similarity between two vectors
    """
    dist1 = float(np.dot(vec1,vec2)/(np.linalg.norm(vec1)*np.linalg.norm(vec2)))
    return dist1

In [ ]:
def coculate_similarity(idx, tmp):
    node_neighbor_list = []
    arch_tuple = tmp[idx]
    arch_idx = arch_tuple[0]
    arch_val = arch_tuple[1]
    
    for next_idx, next_val in tmp[idx+1:]:
        if cos_dist(arch_val, next_val) >= 0.5:
            node_neighbor_list.append(next_idx-1)
    return {arch_idx-1: node_neighbor_list}

In [ ]:
def multiprocess_threads():
    rows = res_matrix[1:].shape[0]
    cols = res_matrix[1:].shape[0]
    mat = sp.lil_matrix((rows, rows), dtype=np.int8) # 73309 means number of articles
    dict_node_neighbor = dict()
    pool = Pool()
    results = []
    tmp_keys = [idx for idx, _ in tmp[:]]
    # 嘗試減少數量
    tmp_keys = sorted(random.sample(tmp_keys, 3))
    #assign task
    for i in tqdm(tmp_keys[1:], desc="coculate_similarity"):
        results.append(pool.apply_async(coculate_similarity, args=(i, tmp[:])))
    
    #all thread finished
    pool.close()
    pool.join()
    #assign result
    for result in tqdm(results,desc="assign result -> dict"):
        dict_node_neighbor.update(result.get())
    #convert_todok_matrix
    for user_id, movie_ids in tqdm(dict_node_neighbor.items(),desc="dict -> sparse matrix"):
        mat[user_id, movie_ids] = 1
    # remove 1st rows and 1st columns because of padding
    return mat.tocsr()

In [ ]:
multiprocess_threads()

In [ ]:
lil = multiprocess_threads()
lil.todense()

In [ ]:
def deserialize(filename):
    #with open(filename, 'rb') as handle:
    # with tf.gfile.Open(filename, 'rb') as handle: # ml warning 不行
    with tf.io.gfile.GFile (filename, 'rb') as handle:
        # 一直出錯試試看
        return pickle.load(handle)

In [ ]:
def load_acr_module_resources(acr_module_resources_path):
    (acr_label_encoders, articles_metadata_df, content_article_embeddings) = \
              deserialize(acr_module_resources_path)

    tf.logging.info("Read ACR label encoders for: {}".format(acr_label_encoders.keys()))
    tf.logging.info("Read ACR articles metadata: {}".format(len(articles_metadata_df)))
    tf.logging.info("Read ACR article content embeddings: {}".format(content_article_embeddings.shape))

    return acr_label_encoders, articles_metadata_df, content_article_embeddings

In [ ]:
acr_module_resources_path = "/home/hengshiou/Documents/chameleon_recsys/adressa" + "/data_transformed/pickles/acr_articles_metadata_embeddings.pickle"
acr_labels_encoder, articles_metadata_df, content_article_embeddings_matrix = load_acr_module_resources(acr_module_resources_path)

In [ ]:
with np.load('B.npz') as X:
    mtx, dist, _, _ = [X[i] for i in ('mtx','dist','rvecs','tvecs')]

In [ ]:
# 讀取 csr_matrix, 也就是 adj matrix 概念
path = "/home/hengshiou/Documents/chameleon_recsys/adressa" + "/graph/sparse_matrix_threadspool.npz"
sparse_matrix = sp.load_npz(path)
adj = vstack(sparse_matrix[:], format='csr')


In [ ]:
# 建立 feature 出來
# e.g. features = sp.vstack((allx, tx)).tolil()
feat_x = content_article_embeddings_matrix

In [ ]:
sparse_matrix.shape

In [ ]:
#For debug
acr_labels_encoder

In [ ]:
articles_metadata_df

## Reshape 練習

In [102]:
t = tf.constant([[[1, 1, 1],
                [2, 2, 2]],
               [[3, 3, 3],
                [4, 4, 4]],
               [[5, 5, 5],
                [6, 6, 6]]])
tf.size(t)


<tf.Tensor 'Size_25:0' shape=() dtype=int32>

In [57]:
tf.reshape(t, [-1])

<tf.Tensor 'Reshape:0' shape=(18,) dtype=int32>

In [76]:
np.random.normal(loc=0, scale=1, size=(8, 4))

array([[-0.09752588, -0.41623343,  0.07513655,  0.44961343],
       [ 0.80553957,  2.37537563,  0.43870845, -0.41147873],
       [ 0.51745362, -1.23249856,  0.38042796, -0.32122999],
       [-1.62302301, -0.16005602, -0.91906196, -0.58537578],
       [ 1.5297895 ,  1.46007447,  0.34930839, -0.66103211],
       [ 0.56105465,  0.89964206,  1.06131516, -0.24052735],
       [ 1.50574699,  1.3766082 , -0.40292682, -1.87089957],
       [-0.89931482,  0.39722655,  0.35787497, -0.31469828]])

In [119]:
A = np.matrix([
    [0, 1, 0, 0],
    [0, 0, 1, 1], 
    [0, 1, 0, 0],
    [1, 0, 1, 0]],
    dtype=float
)
X = np.matrix([
            [i, -i]
            for i in range(A.shape[0])
        ], dtype=float)
X.shape

(4, 2)